In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [9]:
# 데이터 로드

train = pd.read_parquet('./train_pwm.parquet')
train.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,weekend,month,weather,base_time
0,TRAIN_0000000,20220623,1,17,1,106,지방도1112호선,0,0,60.0,...,0,제3교래교,33.427749,126.662335,0,52.0,0,6,3,3
1,TRAIN_0000001,20220728,1,21,2,103,일반국도11호선,0,0,60.0,...,1,KAL사거리,33.504811,126.526240,0,30.0,0,7,3,4
2,TRAIN_0000002,20211010,4,7,2,103,일반국도16호선,0,0,80.0,...,0,상창육교,33.280072,126.362147,0,61.0,1,10,4,1
3,TRAIN_0000003,20220311,0,13,2,107,태평로,0,0,50.0,...,0,서현주택,33.245565,126.566228,0,20.0,0,3,2,2
4,TRAIN_0000004,20211005,6,8,2,103,일반국도12호선,0,0,80.0,...,0,애월입구,33.462677,126.330152,0,38.0,0,10,4,1


In [2]:
# 데이터 로드 part 2

train2 = pd.read_parquet('./train2_pwm.parquet')
train2.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_turn_restricted,end_node_name,end_turn_restricted,target,weekend,month,weather,base_time
0,TRAIN_0000000,20220623,1,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,0,제3교래교,0,52.0,0,6,3,3
1,TRAIN_0000001,20220728,1,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,1,KAL사거리,0,30.0,0,7,3,4
2,TRAIN_0000002,20211010,4,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,0,상창육교,0,61.0,1,10,4,1
3,TRAIN_0000003,20220311,0,13,2,107,태평로,0,0,50.0,...,0,남양리조트,0,서현주택,0,20.0,0,3,2,2
4,TRAIN_0000004,20211005,6,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,0,애월입구,0,38.0,0,10,4,1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 27 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   base_date              int64  
 2   day_of_week            int32  
 3   base_hour              int64  
 4   lane_count             int64  
 5   road_rating            int64  
 6   road_name              object 
 7   multi_linked           int64  
 8   connect_code           int64  
 9   maximum_speed_limit    float64
 10  vehicle_restricted     float64
 11  weight_restricted      float64
 12  height_restricted      float64
 13  road_type              int64  
 14  start_node_name        object 
 15  start_latitude         float64
 16  start_longitude        float64
 17  start_turn_restricted  int32  
 18  end_node_name          object 
 19  end_latitude           float64
 20  end_longitude          float64
 21  end_turn_restricted    int32  
 22  target            

In [10]:
from sklearn.utils import shuffle

tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

C:\Users\soy66\AppData\Local\Temp\ipykernel_13304\489013793.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
C:\Users\soy66\AppData\Local\Temp\ipykernel_13304\489013793.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


(4426522, 16)
(4426522,)
(274695, 16)


In [3]:
from sklearn.utils import shuffle

tr2 = shuffle(train2.sort_values('base_date')[train2['base_date'] < 20220701])
te2 = shuffle(train2.sort_values('base_date')[train2['base_date'] > 20220631])

y_train2 = tr2['target']
X_train2 = tr2.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test2 = te2['target']
X_test2 = te2.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train2.shape)
print(y_train2.shape)
print(X_test2.shape)

C:\Users\soy66\AppData\Local\Temp\ipykernel_13304\2612364628.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr2 = shuffle(train2.sort_values('base_date')[train2['base_date'] < 20220701])
C:\Users\soy66\AppData\Local\Temp\ipykernel_13304\2612364628.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te2 = shuffle(train2.sort_values('base_date')[train2['base_date'] > 20220631])


(4426522, 12)
(4426522,)
(274695, 12)


In [4]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [15]:

LR2 = lgb.LGBMRegressor()


parameters_2 = {'max_depth':[10,15,20],
              'num_leaves':[150, 250,300],
              'learning_rate' : [0.3,0.5, 0.6],
            'n_estimators':[100,200,500]
             }


grid_lr_2 = GridSearchCV(LR2, param_grid=parameters_2, cv=3)


grid_lr_2.fit(X_train2, y_train2)


scores_df = pd.DataFrame(grid_lr_2.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'learning_rate': 0.3, 'max_depth': 10, 'n_est...",0.632181,1,0.632659,0.632325,0.631559
1,"{'learning_rate': 0.3, 'max_depth': 10, 'n_est...",0.632121,2,0.632588,0.632270,0.631505
2,"{'learning_rate': 0.3, 'max_depth': 10, 'n_est...",0.632031,5,0.632497,0.632165,0.631431
3,"{'learning_rate': 0.3, 'max_depth': 10, 'n_est...",0.632024,6,0.632462,0.632187,0.631424
4,"{'learning_rate': 0.3, 'max_depth': 10, 'n_est...",0.631675,14,0.632117,0.631805,0.631104
...,...,...,...,...,...,...
76,"{'learning_rate': 0.6, 'max_depth': 20, 'n_est...",0.629675,60,0.630079,0.629861,0.629087
77,"{'learning_rate': 0.6, 'max_depth': 20, 'n_est...",0.629340,69,0.629751,0.629505,0.628764
78,"{'learning_rate': 0.6, 'max_depth': 20, 'n_est...",0.629035,73,0.629434,0.629181,0.628490
79,"{'learning_rate': 0.6, 'max_depth': 20, 'n_est...",0.628318,79,0.628729,0.628470,0.627755


In [17]:
grid_lr_2.cv_results_

{'mean_fit_time': array([17.27377415, 19.98424729, 22.05270116, 27.31462169, 35.59478887,
        39.1908652 , 60.45834462, 80.01838024, 88.97142895, 15.99088875,
        19.40910395, 21.24519324, 26.79734794, 33.81558267, 37.44188587,
        59.3808821 , 78.18327403, 86.66226904, 16.10493787, 19.68300724,
        21.14313221, 26.92600425, 34.02369277, 37.40862068, 59.61824576,
        77.7012407 , 86.20381252, 14.86823384, 18.94135332, 20.94864241,
        26.02474713, 33.85712679, 38.07219068, 58.92178639, 78.63574147,
        88.27829266, 14.70234315, 18.23921704, 19.69966014, 25.38612215,
        32.99078687, 36.03797364, 58.34463771, 76.22517737, 84.58450278,
        14.67841903, 18.13849918, 19.84028498, 26.57893213, 32.75904377,
        36.17128452, 59.31905707, 75.89008427, 83.73643684, 14.71597195,
        18.44496878, 20.71062239, 25.88977536, 33.5755562 , 37.75437395,
        59.02052275, 78.6792713 , 88.69483415, 14.24456787, 17.84628288,
        19.75416875, 25.15307864, 

In [ ]:
# learning_rate 0.3, max_depth: 10 , n_estimators:100,num_leaves:150 2번째 파라미터값

In [5]:
LR2 = lgb.LGBMRegressor()

In [12]:
LR2 = lgb.LGBMRegressor(random_state = 42,
                           max_depth = 20,
                           num_leaves = 300,
                           learning_rate = 0.5,
                           n_estimators = 100,
                           
                          ).fit(X_train2, y_train2)

pred2 = LR2.predict(X_test2)
mae2 = mean_absolute_error(pred2, y_test2)
print(mae2)
print(LR2.feature_importances_)

6.615150123278584
[8190 2406 2460 3228 1444  991 1391 1362 2212 5410  804    2]


In [18]:
LR2 = lgb.LGBMRegressor(random_state = 42,
                           max_depth = 10,
                           num_leaves = 150,
                           learning_rate = 0.3,
                           n_estimators = 100,
                           
                          ).fit(X_train2, y_train2)

pred2 = LR2.predict(X_test2)
mae2 = mean_absolute_error(pred2, y_test2)
print(mae2)
print(LR2.feature_importances_)

6.62729380017465
[4031 1459 1485 1905  769  577  675  659  867 1849  621    3]


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 2번째 파라미터를 찾는 중...

In [19]:
params_2 ={
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

rfr_2 = RandomForestRegressor()
grid_cv__2 = GridSearchCV(rfr_2, param_grid=params_2, cv=2, n_jobs=-1)
grid_cv__2.fit(X_train2,y_train2)

GridSearchCV(cv=2, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

In [20]:
print(f"Best Param__2: {grid_cv__2.best_params_}")

Best Param__2: {'max_depth': 12, 'min_samples_leaf': 18, 'min_samples_split': 8, 'n_estimators': 100}


In [ ]:
# 첫번째 파라미터들을 사용해서 나온값

In [14]:
rfr_2 = RandomForestRegressor(max_depth=12,min_samples_leaf=8,min_samples_split=20,n_estimators=100).fit(X_train2, y_train2)
pred_2=rfr_2.predict(X_test2)
mae_2=mean_absolute_error(pred_2,y_test2)
print(mae)

6.7750960152607975


In [ ]:
# 2번째 파라미터 바꾸고 나온 값

In [21]:
rfr__2 = RandomForestRegressor(max_depth=12,min_samples_leaf=8,min_samples_split=18,n_estimators=100).fit(X_train2, y_train2)
pred__2=rfr__2.predict(X_test2)
mae__2=mean_absolute_error(pred__2,y_test2)
print(mae__2)

6.775080610393338


In [ ]:
# randomforest 회귀

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
params ={
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

rfr = RandomForestRegressor()
grid_cv = GridSearchCV(rfr, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train,y_train)

In [ ]:
print(f"Best Param: {grid_cv.best_params_}")

In [ ]:
#max_depth=12,min_samples_leaf :8,min_samples_split:20,n_estimators:100

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
rfr = RandomForestRegressor(max_depth=12,min_samples_leaf=8,min_samples_split=20,n_estimators=100).fit(X_train, y_train)
pred=rfr.predict(X_test)
mae=mean_absolute_error(pred,y_test)
print(mae)

In [ ]:
# 선형회귀

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lr= LinearRegression()

In [15]:
lr.fit(X_train,y_train)

LinearRegression()

In [16]:
lr_pred = lr.predict(X_test)

In [17]:
lr_mae = mean_absolute_error(lr_pred, y_test)
print(lr_mae)

10.518630641861185


In [ ]:
# one-hot 인코딩

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
enc=OneHotEncoder()

In [ ]:
# 특정 시간대에 교통량 영향을 주는 지 확인 후 나의생각은 00:00~04:00 까지는 영향을 주지않는다 판단 
#

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV



LR = lgb.LGBMRegressor()


parameters = {'max_depth':[10,15,20],
              'num_leaves':[150, 250,300],
              'learning_rate' : [0.3,0.5, 0.6],
            'n_estimators':[100,200,500]
             }


grid_lr = GridSearchCV(LR, param_grid=parameters, cv=3)


grid_lr.fit(X_train, y_train)


scores_df = pd.DataFrame(grid_lr.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
grid_lr.cv_results_

In [ ]:
#{'learning_rate': 0.5, 'max_depth': 20, 'num_leaves': 300},

In [11]:
# grid_dtree.cv_results_
LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = 20,
                           num_leaves = 300,
                           learning_rate = 0.5,
                           n_estimators = 100,
                           
                          ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)
print(mae)
print(LR.feature_importances_)

3.5964774613238135
[8327 1017  779  862  351  171 3725 2775   97 2828 2270  120 1374 3933
 1265    6]


In [ ]:
importances = pd.Series(LR.feature_importances_, X_train.columns)
importances.sort_values()